In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import seaborn as sns
import tensorflow as tf
from tensorflow.contrib.deprecated import histogram_summary
%matplotlib inline
np.random.seed(20170731)
tf.set_random_seed(20170731)

In [2]:
def plotchr(image,label,i):
    sns.set_context("talk")
    plt.subplot(1,6,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.title("%d" % np.argmax(label))
    plt.imshow(image,cmap=plt.cm.gray_r)

In [3]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


# モデルの作成

In [4]:
num_units = 1024
# モデルの作成
x = tf.placeholder(tf.float32, [None, 784])

w1 = tf.Variable(tf.truncated_normal([784, num_units]))
b1 = tf.Variable(tf.zeros([num_units]))
# h1 = tf.nn.relu(tf.matmul(x,w1)+b1)
h1 = tf.sigmoid(tf.matmul(x,w1)+b1)

w0 = tf.Variable(tf.zeros([num_units,10]))
b0 = tf.Variable(tf.zeros([10]))

h2 = tf.nn.sigmoid(tf.matmul(h1,w0)+b0)
out = tf.nn.softmax(h2)

with tf.name_scope("Histgram") as scope:
    w_hist = tf.summary.histogram("weight",w1)
    b_hist = tf.summary.histogram("biases",b1)
    h2_hist = tf.summary.histogram("h2",h2)
    out_hist = tf.summary.histogram("out",out)

# 誤差関数とオプティマイザーを定義

In [5]:
t = tf.placeholder(tf.float32, [None, 10], name="ideal-output")

with tf.name_scope("Evaluations") as scope:
    loss = -tf.reduce_sum(t*tf.log(out))
    _ = tf.summary.scalar("cross_entropy", loss)
    correct_prediction = tf.equal(tf.argmax(out,1),tf.argmax(t,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    _ = tf.summary.scalar("accuracy", accuracy)

train_step = tf.train.AdamOptimizer().minimize(loss)
#train_step = tf.train.GradientDescentOptimizer(0.003).minimize(loss)

merged = tf.summary.merge_all()

# 学習セッション

In [6]:

with tf.Session() as sess:
    writer = tf.summary.FileWriter("test", sess.graph)
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())

    times = 0
    for _ in range(5000):
        times += 1
        batch_xs,batch_ts = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={x:batch_xs, t:batch_ts})
        if times % 100 == 0:
            summm,loss_val, acc_val = sess.run([merged,loss,accuracy],
                feed_dict={x:mnist.test.images, t:mnist.test.labels})
            writer.add_summary(summm,times)
            if times % 1000 == 0:
                print("Step: {0:d}, Loss: {1:f}, Accuracy: {2:f}".format(times, loss_val, acc_val))

Step: 1000, Loss: 15345.476562, Accuracy: 0.935800
Step: 2000, Loss: 15168.183594, Accuracy: 0.948500
Step: 3000, Loss: 15077.349609, Accuracy: 0.954900
Step: 4000, Loss: 15010.467773, Accuracy: 0.962500
Step: 5000, Loss: 14997.658203, Accuracy: 0.964400


# 96%

# 学習結果を用いて文字認識してみる

In [9]:
plt.figure(figsize=(8,15))
def plotchr(image,label1,label2, i):
    sns.set_context("talk")
    plt.subplot(10,6,i)
    plt.xticks([])
    plt.yticks([])
    plt.title("%d / %d" % (np.argmax(label1),np.argmax(label2)))
    plt.imshow(image,cmap=plt.cm.gray_r)
    
images, labels = mnist.test.images, mnist.test.labels
p_val = sess.run(out, feed_dict={x:images, t:labels})

# 0～9 と判定されたものについて、それぞれ正の例を３つ、負の例を３つ表示する
picnumber = 0
for i in range(10):
    c = 0
    for (image, label, pred) in zip(images, labels, p_val):
        prediction,actual = np.argmax(pred), np.argmax(label)
        if prediction != i:  # 
            continue
        if(c<3 and i == actual) or (c>=3 and i != actual):
            picnumber += 1
            plotchr(image.reshape((28,28)),pred,label,picnumber)
            c += 1
            if c > 5:
                break
            

RuntimeError: Attempted to use a closed Session.